In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
#!conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia


In [4]:
import yaml 
import os 
import torch

print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Check if CUDA is available
print(torch.version.cuda)  # Check CUDA version
print(torch.backends.cudnn.enabled)  # Check if cuDNN is enabled


with open("config/train_t1.yaml", "r") as f:
    config = yaml.safe_load(f)
print(config)

TENSORCORE=False
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU: {gpu_name}")

    # Check compute capability
    compute_capability = torch.cuda.get_device_properties(0).major
    if compute_capability >= 7:  # Tensor Cores require compute capability 7.0+
        print("This GPU supports Tensor Cores.")
        TENSORCORE=True
    else:
        print("This GPU does not support Tensor Cores.")
else:
    print("No GPU detected.")

DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

2.6.0+cpu
False
None
True
{'train_image_dir': None, 'train_mask_dir': None, 'val_image_dir': None, 'val_mask_dir': None, 'out_dir': None, 'out_model_name': None, 'log_dir': None, 'classes': 4, 'classes_names': ['background', 'trees', 'buildings', 'roads'], 'encoder': {'name': 'resnet34', 'weights': 'imagenet', 'in_channels': 3, 'out_channels': 1}, 'decoder': {'name': 'unet', 'in_channels': 3}, 'learning_rate': '1e-4', 'lr_schedule': 'cosine', 'batch_size': 142, 'epochs': 100, 'num_workers': 4, 'train_args': {'HorizontalFlip': 0.5, 'ShiftScaleRotate': {'scale_limit': 0.5, 'rotate_limit': 0, 'shift_limit': 0.1, 'p': 1, 'border_mode': 0}, 'PadIfNeeded': {'min_height': 256, 'min_width': 256, 'always_apply': True, 'border_mode': 0}, 'OneOf_CLAHE_Brightness_Gamma': 0.9, 'OneOf_Sharpen_Blur': 0.9, 'OneOf_HueSaturation': 0.9}, 'validation_args': {'PadIfNeeded': {'min_height': 256, 'min_width': 256, 'always_apply': True, 'border_mode': 0}}}
No GPU detected.


In [5]:
base_dir = "D:\DATA\landcover.ai.v1\output"

config["train_image_dir"] = os.path.join(base_dir, "train","img")
config["train_mask_dir"] = os.path.join(base_dir, "train","mask")
config["val_image_dir"] = os.path.join(base_dir, "val","img")
config["val_mask_dir"] = os.path.join(base_dir, "val","mask")
config["test_image_dir"] = os.path.join(base_dir, "test","img")
config["test_mask_dir"] = os.path.join(base_dir, "test","mask")

config["out_dir"] = os.path.join("output","landcover","models")
config["log_dir"] = os.path.join( "outputs","landcover","logs")
os.makedirs(config["out_dir"], exist_ok=True)
os.makedirs(config["log_dir"], exist_ok=True)

config["out_model_name"] = f'{config["decoder"]["name"]}_{config["encoder"]["name"]}_256_256_4_{config["learning_rate"]}_{config["lr_schedule"]}_142_100_4'

In [6]:
from src.dataset import Dataset
from src.args import get_augmentation,get_preprocessing


x_train_dir = config["train_image_dir"]
y_train_dir = config["train_mask_dir"]

x_valid_dir = config["val_image_dir"]
y_valid_dir = config["val_mask_dir"]

ENCODER=config["encoder"]["name"]
ENCODER_WEIGHTS=config["encoder"]["weights"]


get_preprocessing
train_dataset = Dataset(
    x_train_dir,
    y_train_dir,
    augmentation=get_augmentation(config["train_args"]),
    preprocessing=get_preprocessing(ENCODER,ENCODER_WEIGHTS),
    DEVICE=DEVICE
)

valid_dataset = Dataset(
    x_valid_dir,
    y_valid_dir,
    augmentation=get_augmentation(config["validation_args"]),
    preprocessing=get_preprocessing(ENCODER,ENCODER_WEIGHTS),
    DEVICE=DEVICE
)


c:\Users\jarvi\Documents\GIT\segmentation\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\jarvi\Documents\GIT\segmentation\.conda\Lib\site-packages\albumentations\core\validation.py:45: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
c:\Users\jarvi\Documents\GIT\segmentation\src\args.py:15: UserWarning: Argument 'always_apply' is not valid and will be ignored.
  train_transform.append(albu.PadIfNeeded(**config["PadIfNeeded"]))


In [8]:
from src.visualize import visualize
import numpy as np 

for i in range(3):
    image, mask = train_dataset[np.random.randint(0,train_dataset.__len__())]
    visualize(
        image=image.to('cpu').detach().numpy().copy(), #.transpose(1,2,0),
        mask=mask.to('cpu').detach().numpy().copy().transpose(2,1,0)[-1].transpose(1,0)
    )

AttributeError: 'Dataset' object has no attribute 'DEVICE'

In [7]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------ --------------------------------- 1.3/8.0 MB 7.4 MB/s eta 0:00:01
   ------------------- -------------------- 3.9/8.0 MB 10.7 MB/s eta 0:00:01
   ------------------------------------- -- 7.6/8.0 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 11.8 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 20.6 MB/s eta 0:00:00
